### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `CHAINLIT_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [ ]:
pip install -r ../requirements.txt

In [1]:
pip install -e ..

Obtaining file:///Users/willydouhard/sdks/python-client
  Preparing metadata (setup.py) ... done
  Running setup.py develop for chainlit-client

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Example

In [1]:
import asyncio
import json
import uuid

from chainlit_client import ChainlitClient
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

sdk = ChainlitClient(api_key="foo",batch_size=2)
sdk.instrument_openai()

In [2]:
@sdk.step(type="RUN")
def get_completion(input):
    print("2")
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return completion


@sdk.thread
def thread():
    print("1")
    message = get_completion("hello")

thread()

1
2


IndexError: pop from empty list

In [4]:
@sdk.step(type="RUN")
def get_completion(input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return completion


with sdk.thread():
    message = get_completion("hello")

Failed to send steps: Expecting value: line 1 column 1 (char 0)
Failed to send steps: Expecting value: line 1 column 1 (char 0)


In [5]:
@sdk.step(type="RUN")
def get_completion(input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return completion


with sdk.thread():
    print(1)
    message = get_completion("hello")

1
Failed to send steps: Expecting value: line 1 column 1 (char 0)
Failed to send steps: Expecting value: line 1 column 1 (char 0)


In [8]:
def get_completion(input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return completion


with sdk.thread() as t:
    print(t.id)
    with sdk.step(type="RUN") as s:
        message = get_completion("hello")

9223086a-adb2-42b2-8adc-008493992dbd
8efda7fc-e9d5-4be0-887e-17227344d240
Failed to send steps: Expecting value: line 1 column 1 (char 0)
Failed to send steps: Expecting value: line 1 column 1 (char 0)


In [ ]:
sdk.wait_until_queue_empty() # wait for all msgs to be in the cloud instance

In [ ]:
# Get the thread

print("\nSearching for the thread", thread_id, "...")
thread = await sdk.api.get_thread(id=str(thread_id))

print(json.dumps(thread.to_dict(), indent=2))